In [1]:
import torch
from transformers import pipeline

# 1. Initialize the pipeline
summarizer = pipeline(
    task="text-generation",
    model="google/gemma-3-1b-it",
    device=0,
    torch_dtype=torch.bfloat16
)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-25 16:24:15.533005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769358255.551880   13025 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769358255.558016   13025 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769358255.576301   13025 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00

In [ ]:
import json

input_file = "billsum_v4_1/org_with_cefr_labels/us_test.jsonl"
dataset = [json.loads(line.strip()) for line in open(input_file)]
print(dataset[0]['text'])
dataset = dataset[:100]

In [3]:
import json
from jinja2 import Template

prompt_file = "summarization_prompt.json"
prompt_json = json.loads(open(prompt_file).read().strip())
print(prompt_json)

jinja_template = Template(prompt_json["user_prompt"])
print(jinja_template)

{'system_prompt': 'You are helpful assistant designed to make English legal text more readable for different target audience at different CEFR readability levels.', 'user_prompt': 'Summarize the following text for a {{ level }} reader. {{ text }} \n Please output the summary as a paragraph.'}
<Template memory:705c159ebd00>


In [4]:
# from transformers import pipeline
# from tqdm import tqdm

# cefr_labels = ["A1", "A2", "B1", "B2", "C1", "C2"]

# for i in tqdm(range(len(dataset))):
    
#     text = dataset[i]["text"]
#     messages = [
#             [
#                 {
#                     "role": "system",
#                     "content": [{"type": "text", "text": prompt_json["system_prompt"]},]
#                 },
#                 {
#                     "role": "user",
#                     "content": [{"type": "text", "text": jinja_template.render(text=text, level=label)},]
#                 },
#             ]
#         for label in cefr_labels
#     ]

#     summaries = summarizer(messages, max_new_tokens=512, batch_size=6)

#     all_summaries = {}
#     for summary, label in zip(summaries, cefr_labels):
#         summary_text = summary[-1]["generated_text"][-1]["content"]

#         summary_text = summary_text.split("\n\n")
#         summary_text = "\n\n".join(summary_text[1:])

#         all_summaries[label] = summary_text

#     dataset[i]["predictions"] = all_summaries


In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm

cefr_labels = ["A1", "A2", "B1", "B2", "C1", "C2"]

# 1. Flatten the dataset into "Task Rows"
# Each row in 'flat_data' will represent ONE summary task
flat_data = []
for item in dataset:
    for label in cefr_labels:
        # Pre-render the messages
        msg = [
            {"role": "system", "content": prompt_json["system_prompt"]},
            {"role": "user", "content": jinja_template.render(text=item["text"], level=label)}
        ]
        flat_data.append({"messages": msg})

# 2. Use a Generator to stream data (prevents the 'sequential' warning)
def data_generator():
    for item in flat_data:
        # print(item)
        yield item["messages"]

# 3. Run the pipeline (Properly configured)
# 'batch_size' goes here, NOT in model_kwargs
summaries = []
for out in tqdm(summarizer(data_generator(), batch_size=32, max_new_tokens=512), total=len(flat_data)):
    # Extract the text content from the pipeline output
    summary_text = out[-1]["generated_text"][-1]["content"]
    
    # Cleaning logic (removing the preamble if exists)
    # parts = summary_text.split("\n\n")
    # cleaned = "\n\n".join(parts[1:]) if len(parts) > 1 else summary_text
    summaries.append(summary_text)

# 4. Re-shape the results back into your original 18,948 dataset rows
for i in range(len(dataset)):
    start_idx = i * 6
    end_idx = start_idx + 6
    # Map the 6 sequential summaries back to their CEFR labels
    dataset[i]["predictions"] = dict(zip(cefr_labels, summaries[start_idx:end_idx]))

In [ ]:
print(dataset[0])

In [6]:
from collections import Counter

output_file = "billsum_v4_1/predictions/oob/us_test.jsonl"

with open(output_file, "w") as fp:
    for instance in dataset:
        fp.write(json.dumps(instance).strip() + "\n")

    